라이브러리 불러오기

In [97]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime
import numpy as np

데이터 불러오기

In [98]:
# 착지 파일 업로드
landing_df = pd.read_csv('착지.csv')

# 주문 파일 업로드
order_mon_df = pd.read_csv('주문.csv')

# OD Matrix 파일 업로드
OD_csv = pd.read_csv('OD_Matrix.csv')

# 센터 파일 업로드
center = pd.read_csv('센터.csv')

# 차량 파일 업로드 
vehicle = pd.read_csv('차량.csv')

착지 테이블과 주문 테이블 합치기

In [99]:
# ID와 Stop_ID를 기준으로 병합
integration_mon_df = pd.merge(landing_df[:], order_mon_df[:], left_on='ID', right_on='Stop_ID')

# 불필요한 열 제거 (Y_y, X_y만 제거, Stop_ID는 Order_ID와 함께 유지)
integration_mon_df = integration_mon_df.drop(columns=['Y_y', 'X_y'])

# 열 이름 변경
integration_mon_df = integration_mon_df.rename(columns={'Y_x': 'Y', 'X_x': 'X'})

# 열 순서 재정렬
cols_1 = ['Order_ID', 'ID'] + [col for col in integration_mon_df.columns if col not in ['Order_ID', 'ID']]

integration_mon_df = integration_mon_df[cols_1]

진입제한이 1000인 테이블, 진입제한이 1000이 넘으면서 무게가 3000을 안 넘는 테이블, 진입제한이 1000을 넘으면서 무게가 3000을 넘는 테이블로 나누기

In [100]:
one_mon_df = integration_mon_df[integration_mon_df['Access_restriction'] == 1000]
else_mon_df = integration_mon_df[integration_mon_df['Access_restriction'] != 1000]

In [101]:
three_mon_df = else_mon_df[(else_mon_df['weight'] <= 3500) & (else_mon_df['plt'] <= 8)]
eleven_mon_df = else_mon_df[(else_mon_df['weight'] > 3500) | (else_mon_df['plt'] > 8)]

클러스터링 함수

In [104]:
def clustering(order_df, num_cluster):
    # Step 1: 착지 파일에서 Start_time과 End_time을 기준으로 클러스터링 (4개 클러스터)
    kmeans_time = KMeans(n_clusters=4, random_state=42)
    landing_df['Cluster_time'] = kmeans_time.fit_predict(landing_df[['Start_time', 'End_time']])

    # Step 2: 각 클러스터별로 ID 리스트를 생성
    cluster_id_lists = [[] for _ in range(4)]
    for cluster_num in range(4):
        cluster_id_lists[cluster_num] = landing_df[landing_df['Cluster_time'] == cluster_num]['ID'][:].tolist()

    # Step 3: 주문 파일에서 각 클러스터 리스트의 ID와 Stop_ID를 매칭하여 클러스터별로 데이터 분리 및 클러스터링
    location_cluster_dfs = []
    for cluster_num, id_list in enumerate(cluster_id_lists):
        cluster_time = order_df[order_df['Stop_ID'].isin(id_list)].copy()
        
        # Step 4: 각 클러스터 내에서 X와 Y를 기준으로 다시 클러스터링 (예: num_cluster 개 클러스터)
        kmeans_location = KMeans(n_clusters=num_cluster, random_state=42)
        cluster_time['Cluster_location'] = kmeans_location.fit_predict(cluster_time[['X', 'Y']])
        
        # 각 위치 클러스터별 데이터프레임을 저장할 리스트
        cluster_location_dfs = []
        for inner_cluster_num in range(num_cluster):
            cluster_location_df = cluster_time[cluster_time['Cluster_location'] == inner_cluster_num].copy()
            cluster_location_dfs.append(cluster_location_df)
        
        # 각 시간별 클러스터에 해당하는 위치별 클러스터 데이터프레임을 저장
        location_cluster_dfs.append(cluster_location_dfs)
    
    return location_cluster_dfs

In [105]:
mon_cluster_1 = clustering(one_mon_df, 4)
mon_cluster_3 = clustering(three_mon_df, 9)
mon_cluster_11 = clustering(eleven_mon_df, 3)

착지별 OD Matrix 함수

In [107]:
def OD_Matrix(Bin):
    # OriginID와 DestinationID 리스트
    origin_ids = Bin[:]
    destination_ids = Bin[:]

    # 리스트에 termianl 추가
    origin_ids.append('Terminal')
    destination_ids.append('Terminal')

    # 데이터를 필터링하여 필요한 OriginID와 DestinationID만 남기기
    filtered_data = OD_csv[(OD_csv['OriginID'].isin(origin_ids)) & OD_csv['DestinationID'].isin(destination_ids)]

    # 피벗 테이블을 사용하여 2차원 OD Matrix 만들기 (Total_Time 기준)
    od_time = filtered_data.pivot(index='OriginID', columns='DestinationID', values='Total_Time')
    od_distance = filtered_data.pivot(index='OriginID', columns='DestinationID', values='Total_Distance')

    return od_distance, od_time

Bin 생성 알고리즘

In [108]:
def sort_and_make_bin(cluster, max_weight, max_plt, MaxCount):
    # 각 Stop_ID에 해당하는 Order_ID 리스트와 총 무게, total_plt를 계산하여 데이터프레임 생성
    stop_order_map = cluster.groupby('Stop_ID')['Order_ID'].apply(list).reset_index()
    stop_order_map['total_weight'] = cluster.groupby('Stop_ID')['weight'].sum().values
    stop_order_map['total_plt'] = cluster.groupby('Stop_ID')['plt'].sum().values

    # total_weight를 기준으로 내림차순 정렬하여 무거운 주문부터 처리
    stop_order_map = stop_order_map.sort_values(by='total_weight', ascending=False).reset_index(drop=True)
    
    grouped_orders = []  # 그룹화된 주문을 저장할 리스트
    grouped_stop_ids = []  # 그룹화된 Stop_ID를 저장할 리스트
    group_weights = []  # 각 그룹의 총 무게 리스트 (이차원 리스트)
    group_plts = []  # 각 그룹의 총 팔레트 수 리스트 (이차원 리스트)
    
    current_group = []   # 현재 그룹의 주문을 저장할 리스트
    current_stop_ids = []  # 현재 그룹의 Stop_ID를 저장할 리스트
    current_weight = 0   # 현재 그룹의 총 무게를 저장할 변수
    current_plt = 0      # 현재 그룹의 총 팔레트 수를 저장할 변수
    not_check_orders = list(cluster['Order_ID'])  # 처리되지 않은 주문 ID 리스트
    
    while not_check_orders:  # not_check_orders 리스트가 비어있지 않은 동안 루프를 계속 실행. 즉, 처리되지 않은 주문이 남아 있는 한 계속 실행
        any_order_processed = False # 루프가 한 번이라도 실행되는지 확인하기 위한 플래그 변수 초기화
        
        for idx, row in stop_order_map.iterrows():  # stop_order_map의 각 행을 순회하며 Stop_ID와 Order_ID를 가져옴
            stop_id = row['Stop_ID']
            orders = row['Order_ID']
            
            # 같은 착지의 주문들은 무조건 같은 그룹에 포함
            group_weight = sum(cluster[cluster['Order_ID'].isin(orders)]['weight']) # orders 리스트에 포함된 주문들의 총 무게를 계산하여 group_weight 변수에 저장
            group_plt = sum(cluster[cluster['Order_ID'].isin(orders)]['plt']) # orders 리스트에 포함된 주문들의 총 팔레트 수를 계산하여 group_plt 변수에 저장
            
            if group_weight > max_weight or group_plt > max_plt:
                # 큰 주문들을 나눠서 처리
                order_details = cluster[cluster['Order_ID'].isin(orders)][['Order_ID', 'weight', 'plt']].values.tolist()
                order_details.sort(key=lambda x: (-x[1], -x[2])) # 무게와 팔레트 수로 내림차순 정렬
                
                temp_group = []
                temp_weight = 0
                temp_plt = 0
                
                for order in order_details:
                    if temp_weight + order[1] > max_weight or temp_plt + order[2] > max_plt:
                        if temp_group:
                            grouped_orders.append([temp_group])
                            grouped_stop_ids.append([stop_id])
                            group_weights.append([temp_weight])
                            group_plts.append([temp_plt])
                            temp_group = []
                            temp_weight = 0
                            temp_plt = 0
                    temp_group.append(order[0])
                    temp_weight += order[1]
                    temp_plt += order[2]
                    if order[0] in not_check_orders:
                        not_check_orders.remove(order[0])
                
                if temp_group:
                    grouped_orders.append([temp_group])
                    grouped_stop_ids.append([stop_id])
                    group_weights.append([temp_weight])
                    group_plts.append([temp_plt])
                
                any_order_processed = True
                
            else:
                if (current_weight + group_weight <= max_weight and 
                    current_plt + group_plt <= max_plt and 
                    len(current_stop_ids) + 1 <= MaxCount): # 현재 그룹에 새로운 주문들을 추가했을 때, 최대 무게, 최대 팔레트 수 및 최대 착지 수를 초과하지 않는지 확인
                    current_group.append(orders) # orders 리스트의 주문들을 current_group에 추가
                    current_weight += group_weight 
                    current_plt += group_plt
                    if stop_id not in current_stop_ids: # 현재 그룹에 stop_id가 포함되어 있지 않다면 current_stop_ids에 추가
                        current_stop_ids.append(stop_id)
                    for order in orders:
                        if order in not_check_orders: # 주문이 not_check_orders에 있으면
                            not_check_orders.remove(order) # not_check_orders에서 주문을 제거
                    any_order_processed = True
            
            if current_weight >= max_weight or current_plt >= max_plt or len(current_stop_ids) >= MaxCount: 
                # 현재 그룹의 총 무게가 최대 무게에 도달하거나, 총 팔레트 수가 최대 팔레트 수에 도달했거나, 착지 수가 최대 착지 수에 도달했는지 확인
                break # 이 조건이 참이면, 더 이상 현재 그룹에 주문을 추가하지 않기 위해 반복문을 종료
        
        # 현재 그룹을 저장하고 새로운 그룹 시작
        if any_order_processed: # 이번 루프에서 주문이 추가되었는지 확인해서 만약 하나라도 추가된 경우, 현재 그룹을 저장하고 새로운 그룹을 시작
            stop_weights = [sum(cluster[(cluster['Stop_ID'] == sid) & (cluster['Order_ID'].isin(sum(current_group, [])))]['weight']) for sid in current_stop_ids]
            stop_plts = [sum(cluster[(cluster['Stop_ID'] == sid) & (cluster['Order_ID'].isin(sum(current_group, [])))]['plt']) for sid in current_stop_ids]
            grouped_orders.append(current_group)
            grouped_stop_ids.append(current_stop_ids)
            group_weights.append(stop_weights)
            group_plts.append(stop_plts)
            current_group = [] # current_group을 초기화하여 새로운 그룹을 시작할 준비
            current_stop_ids = [] # current_stop_ids를 초기화하여 새로운 그룹을 시작할 준비
            current_weight = 0 # current_weight 초기화
            current_plt = 0 # current_plt 초기화
            
            # stop_order_map에서 이미 처리된 주문을 제거
            # apply 함수로 각 Order_ID 리스트가 not_check_orders에 없는 경우를 필터링
            stop_order_map = stop_order_map[~stop_order_map['Order_ID'].apply(lambda x: all(order not in not_check_orders for order in x))]
            stop_order_map.reset_index(drop=True, inplace=True) # stop_order_map의 인덱스를 재설정하고, drop=True로 기존 인덱스를 제거
        
        # 무한 루프 방지: 주문이 제거되지 않은 경우 루프를 탈출
        if not any_order_processed:
            break

    # 남은 그룹 추가
    if current_group: # 현재 그룹에 주문이 남아 있는 경우, 이 그룹을 저장
        stop_weights = [sum(cluster[(cluster['Stop_ID'] == sid) & (cluster['Order_ID'].isin(sum(current_group, [])))]['weight']) for sid in current_stop_ids]
        stop_plts = [sum(cluster[(cluster['Stop_ID'] == sid) & (cluster['Order_ID'].isin(sum(current_group, [])))]['plt']) for sid in current_stop_ids]
        grouped_orders.append(current_group)
        grouped_stop_ids.append(current_stop_ids)
        group_weights.append(stop_weights)
        group_plts.append(stop_plts)

    # grouped_orders를 3차원 리스트로 변환 (Stop_ID 기준으로 나누기)
    final_grouped_orders = []
    for group in grouped_orders:
        temp_group = []
        for stop_id_list in group:
            temp_group.append(stop_id_list)
        final_grouped_orders.append(temp_group)

    return final_grouped_orders, grouped_stop_ids, group_weights, group_plts


시간 계산 함수

In [109]:
def min_cal(current_time, add_min = 0, plt = 0):
    # 현재 시간을 시와 분으로 분리
    current_hour = int(current_time)
    current_min = (current_time - current_hour)*100 

    add_time_hour = add_min //60
    add_time_min = add_min % 60 
    
    # plt의 개수당 10분을 추가
    additional_minutes = (plt * 5) % 60 
    additional_houres = (plt * 5) // 60

    # 총 분 계산 (기존 분 + 추가할 분 + plt에 의한 추가 분)
    new_time_hour = current_hour + add_time_hour + additional_houres
    new_time_min = current_min + add_time_min + additional_minutes
    new_time = new_time_hour + (new_time_min // 60) + (new_time_min % 60 /100)
        
    return new_time

이동시간, 이동거리 계산 함수

In [111]:
def several_time(route, od_time):
    path = route.copy()
    moving_time_tmp = 0
    for i in range(len(path) - 1):
        moving_time_tmp += od_time.loc[path[i], path[i + 1]]
    out_time_tmp = od_time.loc[path[0], path[1]]  # 터미널에서 첫 착지로 이동하는데 걸린 시간
    in_time_tmp = od_time.loc[path[-2], path[-1]]  # 마지막 착지에서 터미널로 복귀하는데 걸린 시간
    working_time_tmp = moving_time_tmp - (out_time_tmp + in_time_tmp)  # 움직이는 총 시간
    moving_time = min_cal(0, moving_time_tmp, 0)
    working_time = min_cal(0, working_time_tmp, 0)
    out_time = min_cal(0, out_time_tmp, 0)
    in_time = min_cal(0, in_time_tmp, 0)
    return moving_time, working_time, out_time, in_time

def route_distance(route, od_distance):
    path = route.copy()
    # path.insert(0,'Terminal')
    # path.append('Terminal')
    distance = 0
    for i in range(len(path)-1):
        distance += od_distance.loc[path[i],path[i+1]]
    return distance

하차시간 계산 함수

In [112]:
def unloading_time(stop_id_list, plt_list): # plt에 따른 하차하는데 걸린 시간(분) (고정시간+변동시간)
    tmp = 0
    for i in range(len(stop_id_list) - 1):
        tmp += 5
    for i in range(len(plt_list) - 1):
        tmp += plt_list[i]*10
    unloading_time = tmp
    return unloading_time 

라우팅 알고리즘

In [113]:
# 초기해(근사 최적해)를 구하는 Nearest Neighbor 알고리즘
def nearest_neighbor(matrix):
    start_node = 'Terminal'
    n = len(matrix)
    visited = {node: False for node in matrix.index}
    path = [start_node]
    current_node = start_node
    visited[current_node] = True

    while len(path) < n:
        current_index = current_node
        next_node = None
        min_distance = float('inf')
        
        for node in matrix.index:
            if not visited[node] and matrix.loc[current_index, node] < min_distance:
                min_distance = matrix.loc[current_index, node]
                next_node = node
        
        if next_node is None:
            break
        
        path.append(next_node)
        visited[next_node] = True
        current_node = next_node
    path.append('Terminal')
    return path


# 초기해의 꼬인 경로를 푸는 알고리즘
def two_opt(path, matrix):
    def path_cost(path):
        cost = 0
        for i in range(len(path) - 1):
            cost += matrix.loc[path[i], path[i + 1]]
        return cost

    improved = True
    max_no_improve = 10  # 개선이 없을 때 최대 반복 횟수
    no_improve_count = 0

    while improved and no_improve_count < max_no_improve:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path)):
                if j - i == 1: 
                    continue
                new_path = path[:i] + path[i:j][::-1] + path[j:]
                if path_cost(new_path) < path_cost(path):
                    path = new_path
                    improved = True
                    no_improve_count = 0  # 개선이 있으면 카운터 초기화
        if not improved:
            no_improve_count += 1  # 개선이 없으면 카운터 증가
    return path

시간 제약에 따른 빈 분할

In [114]:
def Bin_Slicing(grouped_orders, grouped_stop_ids, group_weights, group_plts, time_windows): 
    grouped_stop_ids_list = grouped_stop_ids.copy()
    grouped_order_list = grouped_orders.copy()
    group_weights_list = group_weights.copy()
    group_plts_list = group_plts.copy()
    
    new_bin_list = []
    new_order_list = []
    new_weight_list = []
    new_plt_list = []
    
    new_moving_time_list = []
    new_working_time_list = []
    new_out_time_list = []
    new_in_time_list = []
    continue_check = True

    while grouped_stop_ids_list:  # 리스트에 요소가 남아있는 동안 반복
        
        if not grouped_order_list or not group_weights_list or not group_plts_list:
            break

        tmp_orders = grouped_order_list.pop(0)
        tmp_stop_ids = grouped_stop_ids_list.pop(0)  # 첫 번째 원소를 가져와서 제거
        tmp_weights = group_weights_list.pop(0)
        tmp_plts = group_plts_list.pop(0)

        for j in range(len(tmp_stop_ids)-1, -1, -1):
            tmp_orders_range = tmp_orders[:j+1]
            tmp_stop_ids_range = tmp_stop_ids[:j+1]
            tmp_weights_range = tmp_weights[:j+1]
            tmp_plts_range = tmp_plts[:j+1]
            

            od_distance, od_time = OD_Matrix(tmp_stop_ids_range)
            initial_path = nearest_neighbor(od_distance)
            optimized_path = two_opt(initial_path, od_distance)
            time_moving, time_working, time_out, time_in = several_time(optimized_path, od_time) 
            unload_time = unloading_time(tmp_stop_ids_range, tmp_plts_range)
            total_time_working = min_cal(time_working, unload_time, 0)
            

            if total_time_working <= time_windows:
                
                new_order_list.append(tmp_orders[:j+1])
                new_bin_list.append(tmp_stop_ids[:j+1])
                new_weight_list.append(tmp_weights[:j+1])
                new_plt_list.append(tmp_plts[:j+1])

                distance, time = OD_Matrix(tmp_stop_ids[:j+1])
                initial = nearest_neighbor(distance)
                optimized = two_opt(initial, distance)
                new_time_moving, new_time_working, new_time_out, new_time_in = several_time(optimized, time)
                new_unload_time = unloading_time(tmp_stop_ids_range, tmp_plts_range)
                new_total_time_working = min_cal(new_time_working, new_unload_time, 0)

                new_moving_time_list.append(new_time_moving)
                new_working_time_list.append(new_total_time_working)
                new_out_time_list.append(new_time_out)
                new_in_time_list.append(new_time_in)
                
                if j+1 < len(tmp_stop_ids):  # 남은 원소가 있는 경우만 추가
                    grouped_order_list.append(tmp_orders[j+1:])
                    grouped_stop_ids_list.append(tmp_stop_ids[j+1:])
                    group_weights_list.append(tmp_weights[j+1:])
                    group_plts_list.append(tmp_plts[j+1:])
                break
            # else:


    return  new_bin_list, new_order_list, new_weight_list, new_plt_list, new_moving_time_list, new_working_time_list, new_out_time_list, new_in_time_list

In [115]:
def extract_bin(stop_ids, order_ids, weight, plt, moving_time, working_time, out_time, in_time, standard):
    # 새로운 리스트를 생성하기 위한 빈 리스트
    extract_stop = []
    extract_order = []
    extract_weight = []
    extract_plt = []
    extract_moving_time = []
    extract_working_time = []
    extract_out_time = []
    extract_in_time = []
    
    fine_stop = []
    fine_order = []
    fine_weight = []
    fine_plt = []
    fine_moving_time = []
    fine_working_time = []
    fine_out_time = []
    fine_in_time = []

    # order_ids의 각 요소를 순회하면서 길이를 확인
    for i in range(len(order_ids)):
        # order_ids[i]의 첫 번째 리스트의 길이가 1 이하인 경우에만 필터링 작업 수행
        if sum(weight[i]) <= standard:
            extract_stop.append(stop_ids[i])
            extract_order.append(order_ids[i])
            extract_weight.append(weight[i])
            extract_plt.append(plt[i])
            extract_moving_time.append(moving_time[i])
            extract_working_time.append(working_time[i])
            extract_out_time.append(out_time[i])
            extract_in_time.append(in_time[i])
        
        else:
            fine_stop.append(stop_ids[i])
            fine_order.append(order_ids[i])
            fine_weight.append(weight[i])
            fine_plt.append(plt[i])
            fine_moving_time.append(moving_time[i])
            fine_working_time.append(working_time[i])
            fine_out_time.append(out_time[i])
            fine_in_time.append(in_time[i])
    
    return extract_stop, extract_order, extract_weight, extract_plt, fine_stop, fine_order, fine_weight, fine_plt, \
        extract_moving_time, extract_working_time, extract_out_time, extract_in_time, fine_moving_time, fine_working_time, fine_out_time, fine_in_time

In [116]:
def decending_bin(stop_ids, order_ids, weight, plt):
    # weight와 각 리스트들을 결합
    tmp_weight = []
    tmp_plt = []
    
    for i in weight:
        sum_weight = sum(i)
        tmp_weight.append([sum_weight])
    for j in plt:
        sum_plt = sum(j)
        tmp_plt.append([sum_plt])
        
    combined = list(zip(stop_ids, order_ids, tmp_weight, tmp_plt, weight, plt))
    
    # weight 기준 내림차순 정렬
    sorted_combined = sorted(combined, key=lambda x: x[2], reverse=True)
    
    # 각 리스트를 분리하고 리스트로 변환
    sorted_stop_ids = [x[0] for x in sorted_combined]
    sorted_order_ids = [x[1] for x in sorted_combined]  # 이미 3차원 리스트 형식
    sorted_weights = [x[2] for x in sorted_combined]
    sorted_plt = [x[3] for x in sorted_combined]
    sorted_ori_weight = [x[4] for x in sorted_combined]
    sorted_ori_plt = [x[5] for x in sorted_combined]
    
    return sorted_stop_ids, sorted_order_ids, sorted_weights, sorted_plt, sorted_ori_weight, sorted_ori_plt

In [117]:
def reposition_bin(stop_ids, order_ids, sum_weight, sum_plt, ori_weight, ori_plt, max_weight, max_plt):
    
    rep_stop_ids = []
    rep_order_ids = []
    rep_sum_weight = []
    rep_sum_plt = []
    rep_ori_weight = []
    rep_ori_plt = []
    
    rev_stop_ids = stop_ids[::-1]
    rev_order_ids = order_ids[::-1]
    rev_sum_weight = sum_weight[::-1]
    rev_sum_plt = sum_plt[::-1]
    rev_ori_weight = ori_weight[::-1]
    rev_ori_plt = ori_plt[::-1]
    
    tmp_order_ids = order_ids[:]
    tmp_rev_order_ids = rev_order_ids[:]
                    
    check_order_ids = []
    
    for i in range(len(sum_weight)):
            for j in range(len(rev_sum_weight)):
            
                if (sum_weight[i][0] + rev_sum_weight[j][0] <= max_weight and
                    sum_plt[i][0] + rev_sum_plt[j][0] <= max_plt and
                    rev_order_ids[j] not in check_order_ids and
                    order_ids[i] not in check_order_ids and
                    order_ids[i] != rev_order_ids[j]):                    
                    
                    check_order_ids.append(order_ids[i][:])
                    check_order_ids.append(rev_order_ids[j][:])
                    
                    tmp_order_ids[i].extend(tmp_rev_order_ids[j])
                    rep_order_ids.append(tmp_order_ids[i])
                    
                    tmp_stop_ids = stop_ids[:]
                    tmp_rev_stop_ids = rev_stop_ids[:]
                    
                    tmp_stop_ids[i].extend(tmp_rev_stop_ids[j])
                    rep_stop_ids.append(tmp_stop_ids[i])
                    
                    tmp_weight = ori_weight[:]
                    tmp_rev_ori_weight = rev_ori_weight[:]
                    
                    tmp_weight[i].extend(tmp_rev_ori_weight[j])
                    rep_ori_weight.append(tmp_weight[i])
                    
                    tmp_plt = ori_plt[:]
                    tmp_rev_ori_plt = rev_ori_plt
                    ori_plt[i].extend(tmp_rev_ori_plt[j])
                    rep_ori_plt.append(tmp_plt[i])
                    
                    break
                    
                elif (order_ids[i] == rev_order_ids[j] and 
                      order_ids[i] not in check_order_ids and
                      rev_order_ids[j] not in check_order_ids):
                    
                    check_order_ids.append(order_ids[i])
                    rep_order_ids.append(order_ids[i])
                    rep_stop_ids.append(stop_ids[i])
                    rep_ori_weight.append(ori_weight[i])
                    rep_ori_plt.append(ori_plt[i]) 
                    
    return rep_stop_ids, rep_order_ids, rep_ori_weight, rep_ori_plt, check_order_ids

In [118]:
def confirm_bin(stop_ids, order_ids, weights, plts, max_weight, max_plt, time_window, standard):
    
    tmp_stop_ids = stop_ids[:]
    tmp_order_ids = order_ids[:]
    tmp_weights = weights[:]
    tmp_plts = plts[:]
    
    confirmed_stop_ids = []
    confirmed_order_ids = []
    confirmed_weights = []
    confirmed_plts = []
    
    confirmed_moving_time = []
    confirmed_working_time = []
    confirmed_out_time = []
    confirmed_in_time = []
    
    already_check_memory = []
    
    stop_ids, order_ids, weight, plt, moving_time, working_time, out_time, in_time = Bin_Slicing(tmp_order_ids, tmp_stop_ids, tmp_weights, tmp_plts, time_window)
    
    extract_stop, extract_order, extract_weight, extract_plt, fine_stop, fine_order, fine_weight, fine_plt, \
        extract_moving_time, extract_working_time, extract_out_time, extract_in_time, \
            fine_moving_time, fine_working_time, fine_out_time, fine_in_time = extract_bin(stop_ids, order_ids, weight, plt, moving_time, working_time, out_time, in_time, standard)
    
    confirmed_stop_ids.extend(fine_stop)
    confirmed_order_ids.extend(fine_order)
    confirmed_weights.extend(fine_weight)
    confirmed_plts.extend(fine_plt)
    
    confirmed_moving_time.extend(fine_moving_time)
    confirmed_working_time.extend(fine_working_time)
    confirmed_out_time.extend(fine_out_time)
    confirmed_in_time.extend(fine_in_time)
    
    already_check_memory.extend(extract_order)  # 변환: 리스트의 튜플로
    tmp_check_memory = already_check_memory[:]

    for i in range(10):
        
        dec_stop_ids, dec_orders, dec_sum_weights, dec_sum_plts, dec_ori_weights, dec_ori_plts = decending_bin(extract_stop, extract_order, extract_weight, extract_plt)
        
        rep_stop_ids, rep_order_ids, rep_ori_weight, rep_ori_plt, check_order_ids = reposition_bin(dec_stop_ids, dec_orders, dec_sum_weights, dec_sum_plts, dec_ori_weights, dec_ori_plts, max_weight, max_plt)
        
        new_stop_ids, new_order_ids, new_weight, new_plt, new_moving_time, new_working_time, new_out_time, new_in_time= Bin_Slicing(rep_order_ids, rep_stop_ids, rep_ori_weight, rep_ori_plt, time_window)
        
        extract_stop, extract_order, extract_weight, extract_plt, fine_stop, fine_order, fine_weight, fine_plt, \
            extract_moving_time, extract_working_time, extract_out_time, extract_in_time, \
                fine_moving_time, fine_working_time, fine_out_time, fine_in_time = extract_bin(new_stop_ids, new_order_ids, new_weight, new_plt, new_moving_time, new_working_time, new_out_time, new_in_time, standard)
        
        if extract_order == tmp_check_memory:
            
            confirmed_stop_ids.extend(extract_stop)
            confirmed_order_ids.extend(extract_order)
            confirmed_weights.extend(extract_weight)
            confirmed_plts.extend(extract_plt)

            confirmed_moving_time.extend(extract_moving_time)
            confirmed_working_time.extend(extract_working_time)
            confirmed_out_time.extend(extract_out_time)
            confirmed_in_time.extend(extract_in_time)
            
            break
        
        if i == 9:
            
            confirmed_stop_ids.extend(extract_stop)
            confirmed_order_ids.extend(extract_order)
            confirmed_weights.extend(extract_weight)
            confirmed_plts.extend(extract_plt)
            
            confirmed_moving_time.extend(extract_moving_time)
            confirmed_working_time.extend(extract_working_time)
            confirmed_out_time.extend(extract_out_time)
            confirmed_in_time.extend(extract_in_time)
            
        else: 

            confirmed_stop_ids.extend(fine_stop)
            confirmed_order_ids.extend(fine_order)
            confirmed_weights.extend(fine_weight)
            confirmed_plts.extend(fine_plt)
            
            confirmed_moving_time.extend(fine_moving_time)
            confirmed_working_time.extend(fine_working_time)
            confirmed_out_time.extend(fine_out_time)
            confirmed_in_time.extend(fine_in_time)
    
            tmp_check_memory = extract_order  # 변환: 리스트의 튜플로

    return confirmed_stop_ids, confirmed_order_ids, confirmed_weights, confirmed_plts, confirmed_moving_time, confirmed_working_time, confirmed_out_time, confirmed_in_time

프로세스

In [119]:
def final_bin(cluster, num):
    final_df = pd.DataFrame(columns=['Stop_list', 'Order_list', 'weight', 'plt', 'moving_time', 'working_time', 'out_time', 'in_time', 'time_cluster', 'time_start', 'time_end'])
    row = []
    if num == 1:
        veh_ton = 1000
        veh_plt = 2.8
        veh_count = 25
        standard = 500
    elif num == 3:
        veh_ton = 3500
        veh_plt = 8
        veh_count = 15
        standard = 1700
    else:
        veh_ton = 11000
        veh_plt = 16
        veh_count = 5
        standard = 5500
    
    for i in range(4):
        if i == 0:
            time_start = 6
            time_end = 9
            time_window = 3
        elif i == 1:
            time_start = 5
            time_end = 22
            time_window = 17
        elif i == 2:
            time_start = 14
            time_end = 16
            time_window = 2
        else:
            time_start = 9
            time_end = 10
            time_window = 1

        for j in range(len(cluster[i])):
            orders, stops, weights, plts = sort_and_make_bin(cluster[i][j], veh_ton, veh_plt, veh_count)
            confirmed_stop_ids, confirmed_order_ids, confirmed_weights, confirmed_plts, confirmed_moving_time, confirmed_working_time, confirmed_out_time, confirmed_in_time = confirm_bin(stops, orders, weights, plts, veh_ton, veh_plt, time_window, standard)
            
            for k in range(len(confirmed_stop_ids)):
                confirmed_stop_ids[k].insert(0, 'Terminal')
                confirmed_stop_ids[k].append('Terminal')            
                row.append({'Stop_list':confirmed_stop_ids[k], 'Order_list':confirmed_order_ids[k],'weight':confirmed_weights[k], 'plt':confirmed_plts[k], 'moving_time':confirmed_moving_time[k], 'working_time':confirmed_working_time[k], 'out_time':confirmed_out_time[k], 'in_time':confirmed_in_time[k], 'time_cluster':i, 'time_start':time_start, 'time_end':time_end})
    final_df = pd.DataFrame(row)
    
    return final_df

In [120]:
mon1_bin_df = final_bin(mon_cluster_1, 1)
mon3_bin_df = final_bin(mon_cluster_3, 3)
mon11_bin_df = final_bin(mon_cluster_11, 11)

Bin DataFrame의 weight와 plt를 합한 DataFrame 추출

In [121]:
def Reform_Bin(bin_df):
    sum_weight = []
    sum_plt = []
    for i in range(len(bin_df)):
        tmp_weight = sum(bin_df.loc[i, 'weight'])
        sum_weight.append(tmp_weight)
        tmp_plt = sum(bin_df.loc[i, 'plt'])
        sum_plt.append(tmp_plt)

    bin_df['weight'] = sum_weight
    bin_df['plt'] = sum_plt
    return bin_df

In [122]:
new_mon1_bin_df = Reform_Bin(mon1_bin_df[:])
new_mon3_bin_df = Reform_Bin(mon3_bin_df[:])
new_mon11_bin_df = Reform_Bin(mon11_bin_df[:])

차량 배차 알고리즘

In [124]:
def Vehicle_Dicision(df):
    if df['weight'] <= 1000 and df['plt'] <= 2.8:
        return 'one'
    elif df['weight'] <= 2500 and df['plt'] <= 4:
        return 'two'
    elif df['weight'] <= 3500 and df['plt'] <= 8:
        return 'three'
    elif df['weight'] <= 5000 and df['plt'] <= 10:
        return 'five'
    elif df['weight'] <= 11000 and df['plt'] <= 16:
        return 'eleven'

In [125]:
# 차량 배차를 위한 임시 데이터프레임
tmp_veh1 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh2 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh3 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh5 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh11 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])

In [126]:
def Assignment_Vehicle(bin_df, date):
    global tmp_veh1, tmp_veh2, tmp_veh3, tmp_veh5, tmp_veh11
    for i in range(len(bin_df)):
        tmp_bin = bin_df.iloc[i]
        veh_type = Vehicle_Dicision(tmp_bin)
        
        if veh_type == 'one':
            tmp_veh = tmp_veh1
        elif veh_type == 'two':
            tmp_veh = tmp_veh2
        elif veh_type == 'three':
            tmp_veh = tmp_veh3
        elif veh_type == 'five':
            tmp_veh = tmp_veh5
        elif veh_type == 'eleven':
            tmp_veh = tmp_veh11

        out_time = tmp_bin['out_time']
        out_time = (out_time // 1) * 60 + (out_time % 1) * 100
        time_start = tmp_bin['time_start']
        time_start = min_cal(time_start, -out_time, 0)
        time_start = min_cal(time_start, -30, 0)
        moving_time = tmp_bin['moving_time']
        moving_time = (moving_time // 1) * 60 + (moving_time % 1) * 100
        
        moving_time = moving_time + tmp_bin['plt']*10 + 5
        
        if time_start < 5:
            time_start = 5
            moving_time += 30
        time_end = min_cal(time_start, moving_time, 0)
        new_row = pd.DataFrame({'date': date, 'first_route': [tmp_bin['Stop_list']], 'first_order': [tmp_bin['Order_list']], 'first_start': [time_start], 'first_end': [time_end], 'first_plt': [tmp_bin['plt']], 'first_weight': [tmp_bin['weight']]})
        tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)

        # else:
        #     flag1 = False
        #     in_time = tmp_bin['in_time']
        #     in_time = (in_time // 1) * 60 + (in_time % 1) * 100
        #     time_end = tmp_bin['time_end']
        #     time_end = min_cal(time_end, in_time, 0)
        #     moving_time = tmp_bin['moving_time']
        #     moving_time = (moving_time // 1) * 60 + (moving_time % 1) * 100
            
        #     moving_time = moving_time + tmp_bin['plt']*10 + 5         
            
        #     if time_end > 24:
        #         time_end = 24
        #     time_start = min_cal(time_end, -moving_time, 0)
        #     time_start = min_cal(time_start, -30, 0)
        #     for j in range(len(tmp_veh)):
        #         if tmp_veh.iloc[j]['first_end'] < time_start and pd.isnull(tmp_veh.iloc[j]['second_start']):
        #             tmp_veh.at[j, 'second_route'] = tmp_bin['Stop_list']
        #             tmp_veh.at[j, 'second_order'] = tmp_bin['Order_list']
        #             tmp_veh.at[j, 'second_start'] = time_start
        #             tmp_veh.at[j, 'second_end'] = time_end   
        #             tmp_veh.at[j, 'second_plt'] = tmp_bin['plt']
        #             tmp_veh.at[j, 'second_weight'] = tmp_bin['weight']
        #             flag1 = True
        #             break

        #     if not flag1:
        #         flag2 = False
        #         out_time = tmp_bin['out_time']
        #         out_time = (out_time // 1) * 60 + (out_time % 1) * 100
        #         time_start = tmp_bin['time_start']
        #         time_start = min_cal(time_start, -out_time, 0)
        #         time_start = min_cal(time_start, -30, 0)
        #         moving_time = tmp_bin['moving_time']
        #         moving_time = (moving_time // 1) * 60 + (moving_time % 1) * 100
                
        #         moving_time = moving_time + tmp_bin['plt']*10 + 5
                
        #         if time_start < 5:
        #             time_start = 5
        #             moving_time += 30
        #         time_end = min_cal(time_start, moving_time, 0)

        #         for j in range(len(tmp_veh)):
        #             if tmp_veh.iloc[j]['first_start'] > time_end and pd.isnull(tmp_veh.iloc[j]['second_start']):
        #                 tmp_veh.at[j, 'second_route'] = tmp_bin['Stop_list']
        #                 tmp_veh.at[j, 'second_order'] = tmp_bin['Order_list']
        #                 tmp_veh.at[j, 'second_start'] = time_start
        #                 tmp_veh.at[j, 'second_end'] = time_end
        #                 tmp_veh.at[j, 'second_plt'] = tmp_bin['plt']
        #                 tmp_veh.at[j, 'second_weight'] = tmp_bin['weight']
        #                 flag2 = True
        #                 break

        #         if not flag2:
        #             new_row = pd.DataFrame({'date': date, 'first_route': [tmp_bin['Stop_list']], 'first_order': [tmp_bin['Order_list']], 'first_start': [time_start], 'first_end': [time_end], 'first_plt': [tmp_bin['plt']], 'first_weight': [tmp_bin['weight']]})
        #             tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)

        # tmp_veh 데이터프레임을 업데이트된 상태로 유지
        if veh_type == 'one':
            tmp_veh1 = tmp_veh
        elif veh_type == 'two':
            tmp_veh2 = tmp_veh
        elif veh_type == 'three':
            tmp_veh3 = tmp_veh
        elif veh_type == 'five':
            tmp_veh5 = tmp_veh
        elif veh_type == 'eleven':
            tmp_veh11 = tmp_veh
        
    return tmp_veh1, tmp_veh2, tmp_veh3, tmp_veh5, tmp_veh11

In [127]:
mon_veh1, mon_veh2, mon_veh3, mon_veh5, mon_veh11 = Assignment_Vehicle(new_mon1_bin_df, 1)
mon_veh1, mon_veh2, mon_veh3, mon_veh5, mon_veh11 = Assignment_Vehicle(new_mon3_bin_df, 1)
mon_veh1, mon_veh2, mon_veh3, mon_veh5, mon_veh11 = Assignment_Vehicle(new_mon11_bin_df, 1)

In [128]:
veh1_dfs = [mon_veh1]
veh2_dfs = [mon_veh2]
veh3_dfs = [mon_veh3]
veh5_dfs = [mon_veh5]
veh11_dfs = [mon_veh11]

veh1_df = pd.concat(veh1_dfs, ignore_index=True)
veh2_df = pd.concat(veh2_dfs, ignore_index=True)
veh3_df = pd.concat(veh3_dfs, ignore_index=True)
veh5_df = pd.concat(veh5_dfs, ignore_index=True)
veh11_df = pd.concat(veh11_dfs, ignore_index=True)

차량 ID 추가하기

In [129]:
def Naming_ID(df, veh_kg):
    ID_row = []
    for i in range(len(df)):
        ID_row.append(f"Veh{veh_kg}_{i+1}")
    df['VehID'] = ID_row
    return df

In [130]:
veh1_df = Naming_ID(veh1_df, 1000)
veh2_df = Naming_ID(veh2_df, 2500)
veh3_df = Naming_ID(veh3_df, 3500)
veh5_df = Naming_ID(veh5_df, 5000)
veh11_df = Naming_ID(veh11_df, 11000)

In [131]:
vehicle1_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])
vehicle2_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])
vehicle3_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])
vehicle5_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])
vehicle11_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])

차량 결과 테이블 생성

In [132]:
def result_vehicle(veh_df, veh_info, fixed_cost, variable_cost):
    for i in range(len(veh_df)):
        tmp = veh_df.iloc[i]
        flag = pd.isnull(tmp['second_plt'])
        
        if flag:
            od_distance, od_time = OD_Matrix(tmp['first_route'])
            mt, wt, ot, it = several_time(tmp['first_route'], od_time)
            working_time = (mt // 1) * 60 + (mt % 1) * 100
            working_time = working_time + 5 * len(tmp['first_route']) + 10 * tmp['first_plt']
            info_row = pd.DataFrame({
                'VehicleID': [tmp['VehID']],
                'Round': [1],
                'OrderCount': [sum(len(sublist) for sublist in tmp['first_order'])],
                'StopCount': [len(tmp['first_route'])-2],
                'Plt': [tmp['first_plt']],
                'TravelDistance': [route_distance(tmp['first_route'], od_distance)],
                'WorkTime': [working_time],
                'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                'UnloadingTime': [3 * len(tmp['first_route']) + 5 * tmp['first_plt']],
                'WaitingTime': [0],
                'TotalCost': [fixed_cost + (route_distance(tmp['first_route'], od_distance)) * variable_cost],
                'FixedCost': [fixed_cost],
                'VariableCost': [(route_distance(tmp['first_route'], od_distance)) * variable_cost]
            })
            veh_info = pd.concat([veh_info, info_row], ignore_index=True)

        else:
            if tmp['first_start'] < tmp['second_start']:
                # First route
                od_distance, od_time = OD_Matrix(tmp['first_route'])
                mt, wt, ot, it = several_time(tmp['first_route'], od_time)
                working_time = (mt // 1) * 60 + (mt % 1) * 100
                working_time = working_time + 5 * len(tmp['first_route']) + 10 * tmp['first_plt']
                info_row = pd.DataFrame({
                    'VehicleID': [tmp['VehID']],
                    'Round': [1],
                    'OrderCount': [sum(len(sublist) for sublist in tmp['first_order'])],
                    'StopCount': [len(tmp['first_route'])-2],
                    'Plt': [tmp['first_plt']],
                    'TravelDistance': [route_distance(tmp['first_route'], od_distance)],
                    'WorkTime': [working_time],
                    'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                    'UnloadingTime': [5 * len(tmp['first_route']) + 10 * tmp['first_plt']],
                    'WaitingTime': [0],
                    'TotalCost': [fixed_cost + (route_distance(tmp['first_route'], od_distance)) * variable_cost],
                    'FixedCost': [fixed_cost],
                    'VariableCost': [(route_distance(tmp['first_route'], od_distance)) * variable_cost]
                })
                veh_info = pd.concat([veh_info, info_row], ignore_index=True)

                # Second route
                od_distance, od_time = OD_Matrix(tmp['second_route'])
                mt, wt, ot, it = several_time(tmp['second_route'], od_time)
                working_time = (mt // 1) * 60 + (mt % 1) * 100
                working_time = working_time + 5 * len(tmp['second_route']) + 10 * tmp['second_plt']
                info_row = pd.DataFrame({
                    'VehicleID': [tmp['VehID']],
                    'Round': [2],
                    'OrderCount': [sum(len(sublist) for sublist in tmp['second_order'])],
                    'StopCount': [len(tmp['second_route'])-2],
                    'Plt': [tmp['second_plt']],
                    'TravelDistance': [route_distance(tmp['second_route'], od_distance)],
                    'WorkTime': [working_time],
                    'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                    'UnloadingTime': [5 * len(tmp['second_route']) + 10 * tmp['second_plt']],
                    'WaitingTime': [0],
                    'TotalCost': [fixed_cost + (route_distance(tmp['second_route'], od_distance)) * variable_cost],
                    'FixedCost': [fixed_cost],
                    'VariableCost': [(route_distance(tmp['second_route'], od_distance)) * variable_cost]
                })
                veh_info = pd.concat([veh_info, info_row], ignore_index=True)

            else:
                # Fisrt route
                od_distance, od_time = OD_Matrix(tmp['second_route'])
                mt, wt, ot, it = several_time(tmp['second_route'], od_time)
                working_time = (mt // 1) * 60 + (mt % 1) * 100
                working_time = working_time + 5 * len(tmp['second_route']) + 10 * tmp['second_plt']
                info_row = pd.DataFrame({
                    'VehicleID': [tmp['VehID']],
                    'Round': [1],
                    'OrderCount': [sum(len(sublist) for sublist in tmp['second_order'])],
                    'StopCount': [len(tmp['second_route'])-2],
                    'Plt': [tmp['second_plt']],
                    'TravelDistance': [route_distance(tmp['second_route'], od_distance)],
                    'WorkTime': [working_time],
                    'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                    'UnloadingTime': [5 * len(tmp['second_route']) + 10 * tmp['second_plt']],
                    'WaitingTime': [0],
                    'TotalCost': [fixed_cost + (route_distance(tmp['second_route'], od_distance)) * variable_cost],
                    'FixedCost': [fixed_cost],
                    'VariableCost': [(route_distance(tmp['second_route'], od_distance)) * variable_cost]
                })
                veh_info = pd.concat([veh_info, info_row], ignore_index=True)

                # Second Route
                od_distance, od_time = OD_Matrix(tmp['first_route'])
                mt, wt, ot, it = several_time(tmp['first_route'], od_time)
                working_time = (mt // 1) * 60 + (mt % 1) * 100
                working_time = working_time + 5 * len(tmp['first_route']) + 10 * tmp['first_plt']
                info_row = pd.DataFrame({
                    'VehicleID': [tmp['VehID']],
                    'Round': [2],
                    'OrderCount': [sum(len(sublist) for sublist in tmp['first_order'])],
                    'StopCount': [len(tmp['first_route'])-2],
                    'Plt': [tmp['first_plt']],
                    'TravelDistance': [route_distance(tmp['first_route'], od_distance)],
                    'WorkTime': [working_time],
                    'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                    'UnloadingTime': [5 * len(tmp['first_route']) + 10 * tmp['first_plt']],
                    'WaitingTime': [0],
                    'TotalCost': [fixed_cost + (route_distance(tmp['first_route'], od_distance)) * variable_cost],
                    'FixedCost': [fixed_cost],
                    'VariableCost': [(route_distance(tmp['first_route'], od_distance)) * variable_cost]
                })
                veh_info = pd.concat([veh_info, info_row], ignore_index=True)
    return veh_info


In [133]:
vehicle1_result = result_vehicle(veh1_df, vehicle1_info, 145400, 173)
vehicle2_result = result_vehicle(veh2_df, vehicle2_info, 173200, 173)
vehicle3_result = result_vehicle(veh3_df, vehicle3_info, 200000, 237)
vehicle5_result = result_vehicle(veh5_df, vehicle5_info, 234000, 355)
vehicle11_result = result_vehicle(veh11_df, vehicle11_info, 185200, 421)

In [134]:
vehicle_result_dfs = [vehicle1_result, vehicle2_result, vehicle3_result, vehicle5_result, vehicle11_result]
vehicle_result = pd.concat(vehicle_result_dfs, ignore_index=True)

vehicle_result

,VehicleID,Round,OrderCount,StopCount,Plt,TravelDistance,WorkTime,TravelTime,UnloadingTime,WaitingTime,TotalCost,FixedCost,VariableCost
0,Veh1000_1,1,3,1,1.88,56.496268,131.120655,97.320655,18.40,0,155173.854380,145400,9773.854380
1,Veh1000_2,1,2,1,2.57,71.464254,160.968095,120.268095,21.85,0,157763.315996,145400,12363.315996
2,Veh1000_3,1,5,2,1.47,97.830501,210.769155,176.069155,19.35,0,162324.676588,145400,16924.676588
3,Veh1000_4,1,7,3,2.52,119.297900,271.347317,221.147317,27.60,0,166038.536674,145400,20638.536674
4,Veh1000_5,1,3,2,2.07,164.424642,320.661907,279.961907,22.35,0,173845.463030,145400,28445.463030
...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,Veh11000_51,1,1,1,13.77,74.308445,281.481637,128.781637,77.85,0,216483.855520,185200,31283.855520
553,Veh11000_52,1,1,1,13.77,39.493507,221.140585,68.440585,77.85,0,201826.766400,185200,16626.766400
554,Veh11000_53,1,1,1,10.22,80.117924,244.188974,126.988974,60.10,0,218929.646031,185200,33729.646031
555,Veh11000_54,1,1,1,11.39,132.412790,297.850376,168.950376,65.95,0,240945.784633,185200,55745.784633


In [135]:
veh1_order_info = pd.DataFrame(columns=['OrderID', 'VehicleID', 'Sequence', 'StopID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DepartureTime'])
veh2_order_info = pd.DataFrame(columns=['OrderID', 'VehicleID', 'Sequence', 'StopID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DepartureTime'])
veh3_order_info = pd.DataFrame(columns=['OrderID', 'VehicleID', 'Sequence', 'StopID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DepartureTime'])
veh5_order_info = pd.DataFrame(columns=['OrderID', 'VehicleID', 'Sequence', 'StopID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DepartureTime'])
veh11_order_info = pd.DataFrame(columns=['OrderID', 'VehicleID', 'Sequence', 'StopID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DepartureTime'])

In [137]:
def date_time(str_value, start_date='2024/09/01'):
    # str_value를 float으로 변환하고 두 자리 소수점으로 포맷팅
    str_value = round(float(str_value), 2)
    str_value = "{:.2f}".format(str_value)
    
    # 시간을 슬래시로 분리
    time = str_value.split('.')
    if len(time) == 2:
        hour, minute = map(int, time)
        # minute이 60 이상인 경우 올바르게 처리
        if minute >= 60:
            hour += minute // 60
            minute = minute % 60
        time_str = f"{hour:02}/{minute:02}"
    else:
        time_str = f"{int(time[0]):02}/00"
    
    # 최종 datetime 문자열 생성
    date_time_str = start_date + "/" + time_str
    
    return date_time_str

def convert_custom_to_datetime(date_str):
    date_parts = date_str.split('/')
    
    # 시간과 분을 올바르게 분리하고 정수로 변환
    year, month, day, hour, minute = map(int, date_parts)
    
    # datetime 객체 생성
    return datetime(year, month, day, hour, minute)

def result_order(veh_df, order_info):
    def calculate_unloading_time(order_id, day):
        if day == 1:
            df = order_mon_df
        # 주문 ID가 데이터프레임에 있는지 확인
        if order_id in df['Order_ID'].values:
            return df[df['Order_ID'] == order_id]['plt'].values[0] * 5 + 3
        else:
            return 0  # 기본값을 0으로 설정 (또는 원하는 다른 값으로 설정)

    def process_route(route, orders, start_time, sequence_start=1):
        nonlocal order_info  # nonlocal 키워드를 사용하여 외부 변수 참조
        od_distance, od_time = OD_Matrix(route)
        arrival_time = min_cal(start_time, 30, 0)
        sequence = sequence_start
        for j in range(1, len(route)-1):
            moving_time = od_time[route[j-1]][route[j]]
            arrival_time = min_cal(arrival_time, moving_time, 0)
            for k in range(len(orders[j-1])):
                unloading_time = calculate_unloading_time(orders[j-1][k], tmp['date'])
                
                at = date_time(arrival_time)
                dt = date_time(min_cal(arrival_time, unloading_time, 0))

                info_row = pd.DataFrame({
                    'OrderID': [orders[j-1][k]],
                    'VehicleID': [tmp['VehID']],
                    'Sequence': [sequence],
                    'StopID': [route[j]],
                    'ArrivalTime': [convert_custom_to_datetime(at)],
                    'WaitingTime': [0],
                    'UnloadingTime': [unloading_time],
                    'DepartureTime': [convert_custom_to_datetime(dt)]
                })
                order_info = pd.concat([order_info, info_row], ignore_index=True)
                arrival_time = min_cal(arrival_time, unloading_time, 0)
                sequence += 1
        return order_info, sequence

    for i in range(len(veh_df)):
        tmp = veh_df.iloc[i]
        flag = pd.isnull(tmp['second_plt'])
        
        if flag:
            order_info, _ = process_route(tmp['first_route'], tmp['first_order'], tmp['first_start'])
        else:
            if tmp['first_start'] < tmp['second_start']:
                order_info, sequence = process_route(tmp['first_route'], tmp['first_order'], tmp['first_start'])
                order_info, _ = process_route(tmp['second_route'], tmp['second_order'], tmp['second_start'], sequence_start=sequence + 1)
            else:
                order_info, sequence = process_route(tmp['second_route'], tmp['second_order'], tmp['second_start'])
                order_info, _ = process_route(tmp['first_route'], tmp['first_order'], tmp['first_start'], sequence_start=sequence + 1)

    return order_info


In [138]:
order1_result = result_order(veh1_df, veh1_order_info)
order2_result = result_order(veh2_df, veh2_order_info)
order3_result = result_order(veh3_df, veh3_order_info)
order5_result = result_order(veh5_df, veh5_order_info)
order11_result = result_order(veh11_df, veh11_order_info)

In [140]:
order_result_dfs = [order1_result, order2_result, order3_result, order5_result, order11_result]
order_result = pd.concat(order_result_dfs, ignore_index=True)

order_result

,OrderID,VehicleID,Sequence,StopID,ArrivalTime,WaitingTime,UnloadingTime,DepartureTime
0,O_627,Veh1000_1,1,S_750,2024-09-01 06:17:00,0,9.60,2024-09-01 06:26:00
1,O_2317,Veh1000_1,2,S_750,2024-09-01 06:26:00,0,4.90,2024-09-01 06:31:00
2,O_2318,Veh1000_1,3,S_750,2024-09-01 06:31:00,0,3.90,2024-09-01 06:35:00
3,O_2528,Veh1000_2,1,S_506,2024-09-01 06:26:00,0,5.85,2024-09-01 06:32:00
4,O_2529,Veh1000_2,2,S_506,2024-09-01 06:32:00,0,13.00,2024-09-01 06:45:00
...,...,...,...,...,...,...,...,...
2756,O_214,Veh11000_51,1,S_252,2024-09-01 08:55:00,0,71.85,2024-09-01 10:07:00
2757,O_646,Veh11000_52,1,S_772,2024-09-01 08:53:00,0,71.85,2024-09-01 10:05:00
2758,O_171,Veh11000_53,1,S_201,2024-09-01 08:50:00,0,54.10,2024-09-01 09:44:00
2759,O_860,Veh11000_54,1,S_1018,2024-09-01 09:09:00,0,59.95,2024-09-01 10:09:00


In [141]:
import os

user_home = os.path.expanduser('~')
desktop_path = os.path.join(user_home, 'Desktop')

order_path = os.path.join(desktop_path, '주문 결과 테이블.csv')
vehicle_path = os.path.join(desktop_path, '차량 결과 테이블.csv')

order_result.to_csv(order_path, index=False)
vehicle_result.to_csv(vehicle_path, index=False)
